In [1]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile

# URL to the PRISM 2025 daily precipitation data
BASE_URL = "https://data.prism.oregonstate.edu/daily/ppt/2025/"
DOWNLOAD_DIR = "prism_2025_ppt"
UNZIP_DIR = os.path.join(DOWNLOAD_DIR, "unzipped")

# Create directories if they don't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(UNZIP_DIR, exist_ok=True)

# Step 1: Scrape the file list from the webpage
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Step 2: Find all .zip links
file_links = [a['href'] for a in soup.find_all('a', href=True)
              if a['href'].endswith('.zip')]

# Step 3: Download and unzip each file
for file_name in (file_links):
    file_url = BASE_URL + file_name
    local_zip_path = os.path.join(DOWNLOAD_DIR, file_name)

    # Download file if not already present
    if not os.path.exists(local_zip_path):
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(local_zip_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

    # Unzip file
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(UNZIP_DIR)